In [1]:
import numba
from numba import cuda
import numpy as np
import math

In [2]:
# CONSTANTS
ATOM_MAX = 29
N = int(5e8)

In [3]:
shape = ( N, ATOM_MAX, ATOM_MAX )

In [4]:
@cuda.jit
def distance_mat(crds, natoms, edge_feats):
    
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bx = cuda.blockIdx.x
    
    edge_feats[bx, tx, ty] = 0.0
    
    # bounds check
    N = natoms[bx]
    if tx >= N: return
    if ty >= N: return 
    
    for i in range(3):
        edge_feats[bx, tx, ty] += ( crds[bx, tx, i] - crds[bx, ty, i] )**2
    edge_feats[bx, tx, ty] = math.sqrt(edge_feats[bx, tx, ty])  



In [ ]:
# create result array on the GPU
edge_feats_gpu = cuda.device_array(shape, dtype=np.float32)

# transfer structures data (the coordinates) to GPU
structures_gpu = cuda.to_device(structures_crd_mat)
natoms_gpu = cuda.to_device(structures_natoms_mat)

# launch kernel
distance_mat[ N , (29, 29)](structures_gpu, natoms_gpu, edge_feats_gpu)

# copy the data back
edge_feats_cpu = edge_feats_gpu.copy_to_host()